In [1]:
# !pip install openai-whisper
# !pip install firebase-admin
# !pip install google-api-python-client

In [2]:
#terminal - python show ray

In [3]:
import os
import time
import datetime
import json
import urllib.parse as p
import subprocess
import requests
import re
import warnings
import sys
import math


import youtube_dl
from pytube import YouTube  # !pip install pytube
from pytube.exceptions import RegexMatchError
from tqdm.auto import tqdm  # !pip install tqdm
# from youtube_transcript_api import YouTubeTranscriptApi



/Users/kitanademidun/anaconda3/envs/wordtoobe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### whisper model set up

In [4]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import ray
import whisper
import torch
audio_file= open("audio_file.mp3", "rb")
device = "cuda" if torch.cuda.is_available() else "cpu"
# model = whisper.load_model("base").to(device)
# result = model.transcribe('audio_file.mp3')
# result

2023-08-13 20:24:54,157	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/Users/kitanademidun/anaconda3/envs/wordtoobe/lib/python3.11/site-packages/whisper/timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit


#### google client set-up

In [5]:
import googleapiclient.discovery
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = os.getenv('GOOGLE_DEVELOPER_KEY')

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = DEVELOPER_KEY)

#### firebase set-up

In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('{}.json'.format(os.getenv('PROJECT_ID')))

app = firebase_admin.initialize_app(cred)

db = firestore.client(app)
batch = db.batch()

### Functions

#### helper functions

In [7]:
class TeeToFile:
    """
    In the code , the TeeToFile class is designed to mimic the behavior of the tee command in Unix-like systems. It captures the output and sends it both to the original sys.stdout (the console) and to a specified file. You can instantiate this class with the desired filename and mode (e.g., 'w' for write, 'a' for append).

    Once you run this code, any text printed using the print function will be displayed in the notebook and simultaneously saved to the specified file. The TeeToFile instance must be closed using the close method when you're done with it to properly restore the original sys.stdout and close the file.

    Remember that using this approach can lead to some issues, particularly in more complex scenarios. If you encounter unexpected behavior, you might want to consider using a logging library like logging for better control and flexibility.

    """

    def __init__(self, filename, mode='w'):
        self.file = open(filename, mode)
        self.stdout = sys.stdout
        sys.stdout = self

    def __del__(self):
        self.close()

    def write(self, data):
        self.stdout.write(data)
        self.file.write(data)

    def flush(self):
        self.stdout.flush()
        self.file.flush()

    def close(self):
        if self.file:
            sys.stdout = self.stdout
            self.file.close()
            self.file = None


#### functions calling to database

In [8]:
def get_all_channels_from_db():
    """
    Retrieve list of channels from firebase
    """
    docs = db.collection(u'channels').stream()
    channelsCollection = {}
    for doc in docs:
        channelsCollection[doc.id]=doc.to_dict()
    return channelsCollection

def get_all_videos_for_channel_from_db(channelId):
    """
    Retrieve list of videos from firebase
    """

    docs = db.collection(u"videos").order_by('publishedAt', direction=firestore.Query.DESCENDING).where(u"channelId",u"==",u"{}".format(channelId)).stream()

    collection = []
    for doc in docs:
        vid =doc.to_dict()
        collection.append(vid)
    return collection

def get_all_transcripts_for_channel_from_db(channelId):
    """
    Retrieve list of videos from firebase
    """

    docs = db.collection(u"WhisperTranscriptions").where(u"channelId",u"==",u"{}".format(channelId)).stream()

    collection = []
    for doc in docs:
        transcript =doc.to_dict()
        collection.append(transcript)
        
    return collection

def get_channel_doc(channel_id,allChannels):
    #channel_id = video_url_to_channel_id(video_url)

    for record in allChannels.values():
        
        if record['channelId'] == channel_id:
            print("Channel {} already exists.".format(record['title']))
            return record

    channelToAdd = generate_channel_json(video_url, source="id")


    add = input("Create channel for url ? (0 for no, any other key for yes)".format(channelToAdd['title']))

    if add == "0":
        print("No channel for url")
        return


    docRef = db.collection(u"channels").document(u"{}".format(channelToAdd['channelId'])).set(channelToAdd)
    print("Channel {} is created.".format(channelToAdd['title']))
    return channelToAdd

def add_video_to_db(video):
    docRef = db.collection(u"videos").document(u"{}".format(video['videoId']))
    docRef.set(video)


#### function for pulling youtube data from apis

In [9]:
def video_url_to_channel_id(video_url):
    """
    Take a url for a youtube video and get the channel id
    """
    video_param_from_url = video_url.split("/watch?v=")[1].split("&")[0]
    request = youtube.videos().list(
            part="snippet,contentDetails",
            id=video_param_from_url
        )
    response = request.execute()
    channel_id = response['items'][0]['snippet']['channelId']
    return channel_id

def get_uploads_id(youtube, channel_id):
    request = youtube.channels().list(
            part="snippet,contentDetails",
            id=channel_id
        )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_uploaded_videos_response(youtube, channel_id):

    ## define parameters needed
    playlist_id = get_uploads_id(youtube, channel_id) #Plalylist id will get ut the id that returns the full video list

    params = {
        "part":"snippet,contentDetails",
        "playlistId": playlist_id,
        "maxResults": 50
    }


    uploaded_videos_content_details_list = []

    while True:
        response = youtube.playlistItems().list(**params).execute()

        uploaded_videos_content_details_list = uploaded_videos_content_details_list + response.get('items')
        if 'nextPageToken' in response.keys():
            params['pageToken'] = response['nextPageToken']
            continue
        else:
            break

    print("{} videos retreived.".format(len(uploaded_videos_content_details_list)))
    return uploaded_videos_content_details_list

def filter_uploaded_videos_response(youtube, uploaded_videos_response,**args):
    
    max_total=args.get("max_total") if args.get("max_total") else False #max_total will allow us to define the number of videos we want returned
    oldest_date=args.get("oldest_date") if args.get("oldest_date") else False #if a value is provided, only get videos on or after this date

    if oldest_date:
        recent_uploaded_videos_response = list(filter(lambda x:x['snippet']['publishedAt'].split("T")[0] >= '2023-06-30',uploaded_videos_response))
        if max_total:
            return recent_uploaded_videos_response[:max_total]
    elif max_total:
        return uploaded_videos_response[:max_total]






#### functions for getting data into the right format

In [10]:
 
def video_response_to_dict(x):

    video_df = {}
    video_df['id'] = x['id']
    video_df['videoId'] = x['contentDetails']['videoId']
    video_df['title'] = x['snippet']['title']
    video_df['description'] = x['snippet']['description']
    video_df['thumbnail'] = check_nested_dict_keys(x,["snippet","thumbnails","maxres","url"])
    video_df['channelId'] = x['snippet']['channelId']
    video_df['videoId'] = x['contentDetails']['videoId']
    video_df['publishedAt'] =x['snippet']['publishedAt'].split("T")[0]
    video_df['videoUrl'] = "https://www.youtube.com/watch?v={}".format(x['contentDetails']['videoId'] )
    video_df['duration'] = getYoutubeDuration(x['contentDetails']['videoId']  )
    video_df['transcript_status'] = None
    video_df['text'] = ''

    return video_df      
  
    
def video_response_list_to_list_of_dicts(response_list):

    list_of_dicts = []
    for x in response_list:
        if "#shorts" in x['snippet']['title'].lower():
            continue

        video_df = video_response_to_dict(video_response(x))
        list_of_dicts.append(video_df)


    return list_of_dicts      

def generate_channel_json(sourceValue, source="url"):

    if source=="id":
        channel_id = sourceValue
    elif source=="url":
        video_url = sourceValue
        channel_id = video_url_to_channel_id(video_url)
    else:
        raise Exception("{} not a valid source.".format(source))

    request = youtube.channels().list(
            part="snippet,contentDetails",
            id=channel_id
        )
    
    response = request.execute()

    channel_df = {}
    channel_df['channelId'] = channel_id
    channel_df['title'] = response['items'][0]['snippet']['title']
    channel_df['uploads'] = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    channel_df['description'] = response['items'][0]['snippet']['description']
    channel_df['thumbnail'] = check_nested_dict_keys(response['items'][0],["snippet","thumbnails","high","url"])
    channel_df['videos'] = []

    return channel_df



#### functions for extracting details from response

In [11]:
def getYoutubeDuration(videoId):
    responseVideoDetails = youtube.videos().list( part="contentDetails",id=videoId).execute()
    durationResponse=responseVideoDetails['items'][0]['contentDetails']['duration']

    duration_string = durationResponse.replace('PT',"")
    number_values = re.findall('\d+',duration_string)
    symbols_available= ''.join([i for i in duration_string if not i.isdigit()])
    symbol_map = {}
    for symbol in 'HMS':
        index = symbols_available.find(symbol)
        if index > -1:
            symbol_map[symbol] = number_values[index]

    duration = 0


    for idx in symbol_map:
        if idx == "H":
            duration = int(symbol_map[idx])*60*60 + duration
        if idx == "M":
            duration = int(symbol_map[idx])*60 + duration
        if idx == "S":
            duration = int(symbol_map[idx]) + duration

    return duration

def check_nested_dict_keys(dic, keys):
    d = dic
    for key in keys:
        if key in d.keys():
            d = d[key]
            continue
        else:
            return dic['snippet']['thumbnails']['default']['url']

    return d


#### transcription functions

In [12]:
def get_audio(url,_id):
    try:
        yt = YouTube(url)
        video = yt.streams.filter(only_audio=True).first()
        out_file=video.download(output_path="audio_files")
        base, ext = os.path.splitext(out_file)
        new_file = 'audio_files/'+_id+'.mp3'
        os.rename(out_file, new_file)
        a = new_file
    except Exception as e: # work on python 3.x
        print('Could not download data for {}: '.format(_id),e)
        return None
    return a

def transcribe_youtube_video(videoId, videoUrl,model):

    result = {}

# try:
    start_time = time.time()

    audio_downloaded = get_audio(videoUrl,videoId)
    if not audio_downloaded:
        print('Could not get transcription data {} Audio could not download.'.format(videoId))
        return

    print('Downloaded {}'.format(videoId))

    # transcribe to get speech-to-text data
    result = model.transcribe('audio_files/{}.mp3'.format(videoId))
    os.remove("audio_files/{}.mp3".format(videoId))
    source = "whisper"
    time_to_transcbribe = time.time() - start_time
    print('Transcribed {} in {}'.format(videoId,time_to_transcbribe))

#     except Exception as e: # work on python 3.x
#         print('Could not get transcription data {}.'.format(videoId),e)

    return result

def transcription_response_to_json(transcriptResponse, videoId, channelId):
    transcriptObject = {}
    transcriptObject['videoId'] = videoId
    transcriptObject['channelId'] = channelId

    transcriptObject['text'] = transcriptResponse['text']
    segments = transcriptResponse['segments']
    keys = ['start','end','text','id','seek']
    transcriptObject['sentences'] = [{ keep: item[keep] for keep in keys } for item,i in zip(segments,range(len(segments)) )]
    
    return transcriptObject

## Action Oriented Functions

In [13]:
def update_video_db_for_channel(channel_id):
    all_videos_for_channel_in_db = get_all_videos_for_channel_from_db(channel_id)
    uploaded_videos_response = get_uploaded_videos_response(youtube, channel_id)
    cleaned_video_response = [video_response_to_dict(x) for x in uploaded_videos_response]

    video_ids_in_db = [x['videoId'] for x in all_videos_for_channel_in_db ]
    videos_not_in_db = list(filter(lambda x:x['videoId'] not in video_ids_in_db, cleaned_video_response))
    
    print("New videos: ",len(videos_not_in_db))

    for video in videos_not_in_db:    
        add_video_to_db(video)

In [14]:
def find_videos_to_transcribe(channel_id):   
    all_videos_for_channel_in_db =  get_all_videos_for_channel_from_db(channel_id)
    transcripts_for_channel_from_db = get_all_transcripts_for_channel_from_db(channel_id)
    transcript_ids_in_db = [x['videoId'] for x in transcripts_for_channel_from_db ]
    videos_wo_transcript = list(filter(lambda x:x['videoId'] not in transcript_ids_in_db, all_videos_for_channel_in_db))
    videos_for_transcription = [x for x in videos_wo_transcript if (x['duration'] > 60*5) & (x['duration']<60*45) ]
    print("Videos to be transcribed: ",len(videos_for_transcription))
    return videos_for_transcription

In [15]:
def run_transcription_batch( channelId, videos_for_transcription ): 
    model = whisper.load_model("base").to(device)
    warnings.filterwarnings("ignore")
#     tee_to_file = TeeToFile("logs/transcription_logs/transcription_log_{}_{}.csv".format(channelId, 
#                                                                                          datetime.datetime.now().timestamp()), 
#                             mode='a')
    

    for video in videos_for_transcription:
        try:
            print("Transcribing.... {} ".format(video['videoId']))
            print("Duration {} {} ".format(video['videoId'],video['duration']))




            whisper_transcription = transcribe_youtube_video(video['videoId'],
                                                             "https://www.youtube.com/watch?v={}".format( video['videoId'] ),model)

            transcription_json = transcription_response_to_json(whisper_transcription, video['videoId'], channelId)

            docRef = db.collection(u"WhisperTranscriptions").document(u"{}".format(video['videoId']))
            docRef.set( transcription_json)
            videoRef = db.collection(u"videos").document(video['videoId']).update({"transcript_status": True,'text':transcription_json['text']})

            print("Transcription added: {}".format(video['videoId']))

        except:
            print("err {}", video['videoId'])
            continue



#     tee_to_file.close()
    warnings.filterwarnings("default")

## User input

In [16]:
channel_url = "https://www.youtube.com/watch?v=lBCOOTyU46M&t=638s&ab_channel=ColinandSamir"

In [17]:
allChannels = get_all_channels_from_db()

In [18]:
channel_id = video_url_to_channel_id(channel_url)
channel = get_channel_doc(channel_id,allChannels)

Channel Colin and Samir already exists.


In [19]:
all_videos_for_channel_in_db =  get_all_videos_for_channel_from_db(channel_id)

In [20]:
update_video_db_for_channel(channel_id)

382 videos retreived.
New videos:  0


In [77]:
videos_for_transcription = find_videos_to_transcribe(channel_id)

Videos to be transcribed:  50


In [81]:
j=4
l = [(channel_id,videos_for_transcription[i*j:(i+1)*j]) for i in range(math.ceil(len(videos_for_transcription)/j))]

In [ ]:
import concurrent.futures

def your_function(param1, param2):
    # Your function's code here
    result = param1 + param2
    return result

param_sets = l#[(1, 2), (3, 4), (5, 6), (7, 8)]  # Example parameter sets

start = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(run_transcription_batch, *zip(*param_sets))
print("duration =", time.time() - start)

Transcribing.... 3QZkjsMfj3I 
Duration 3QZkjsMfj3I 1207 
Transcribing.... 88067BiKU4Y 
Duration 88067BiKU4Y 1374 
Transcribing.... VbNIh88Nq5k 
Duration VbNIh88Nq5k 2449 
Transcribing.... fyh2-2qXSLI 
Duration fyh2-2qXSLI 1313 
Downloaded 3QZkjsMfj3I
Downloaded fyh2-2qXSLI
Downloaded VbNIh88Nq5k
Downloaded 88067BiKU4Y
Transcribed 3QZkjsMfj3I in 202.0113627910614
Transcription added: 3QZkjsMfj3I
Transcribing.... cbBxEmGOfk4 
Duration cbBxEmGOfk4 1396 
Could not download data for cbBxEmGOfk4:  cbBxEmGOfk4 is age restricted, and can't be accessed without logging in.
Could not get transcription data cbBxEmGOfk4 Audio could not download.
err {} cbBxEmGOfk4
Transcribing.... gGBCbswZbnI 
Duration gGBCbswZbnI 1856 
Downloaded gGBCbswZbnI
Transcribed fyh2-2qXSLI in 278.0321719646454
Transcription added: fyh2-2qXSLI
Transcribing.... gbDunxRfbgg 
Duration gbDunxRfbgg 1390 
Downloaded gbDunxRfbgg
Transcribed 88067BiKU4Y in 298.7525990009308
Transcription added: 88067BiKU4Y
Transcribing.... jzMsnNx